In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from pymongo import MongoClient
from spacy.en import English

In [3]:
import textacy

In [4]:
client = MongoClient('localhost', 27017)
db = client.python_import
collection = db.earnings_transcript_NAS100

In [6]:
def remove_non_ascii(text):
    return text.encode('utf-8', 'ignore').decode('utf-8')

In [7]:
def export_transcripts_to_corpus(transcripts, name='NAS100'):
    texts = transcripts['rawText'].apply(remove_non_ascii).tolist()
    metadatas = transcripts['_id'].apply(str).tolist()
    
    step = 1000
    for i in range(0, len(texts), step):
        _from = i
        _to = i+step if (i+step+1 < len(texts)) else len(texts) - 1
        corpus = textacy.Corpus('en', texts=texts[_from:_to], metadatas=metadatas[_from:_to])
        print('{} - {} added'.format(i, _to))
        corpus.save('./data', name='earnings_transcript_' + name + '_' + str(_to), compression='gzip')

In [10]:
transcripts = pd.DataFrame(list(collection.find({'tradingSymbol':'AAPL'})))

In [11]:
export_transcripts_to_corpus(transcripts, 'AAPL')

0 - 42 added


In [142]:
corpus_aapl = textacy.Corpus.load('./data', name='earnings_transcript_AAPL_42', compression='gzip')

In [148]:
print(corpus_aapl)

Corpus(42 docs; 421447 tokens)


# Tf-Idf calculation

In [157]:
from sklearn.feature_extraction.text import CountVectorizer

In [181]:
nlp = English()
def tokenize(text):
    doc = nlp(text)
    return [tok.lemma_ for tok in doc if tok.is_alpha and not tok.is_stop]

In [174]:
count_model = CountVectorizer(ngram_range=(1,1))

In [176]:
lemmatized_words = [[tok.lemma_ for tok in spacy_doc if tok.is_alpha and not tok.is_stop] for spacy_doc in corpus_aapl]

In [179]:
len(lemmatized_words)

42

In [180]:
[' '.join(lemmatized for lemmatized in lemmatized_words)]

TypeError: sequence item 0: expected str instance, list found

In [169]:
X = count_model.fit_transform([tok.lemma_ for tok in spacy_doc] for spacy_doc in corpus_aapl)

AttributeError: 'list' object has no attribute 'lower'

In [168]:
type(corpus_aapl.docs)

spacy.tokens.token.Token

# Co-occurrance

In [64]:
co_occurrence = doc_term_matrix.T*doc_term_matrix

In [65]:
co_occurrence.setdiag(0)

In [66]:
co_occurrence.todense()

matrix([[   0.        ,   12.13142472,  121.63103692, ...,    0.        ,
            0.        ,    0.        ],
        [  12.13142472,    0.        ,   29.12640369, ...,   39.53326795,
            7.90665359,   31.62661436],
        [ 121.63103692,   29.12640369,    0.        , ...,    0.        ,
            0.        ,    0.        ],
        ..., 
        [   0.        ,   39.53326795,    0.        , ...,    0.        ,
          211.88381197,  463.49583869],
        [   0.        ,    7.90665359,    0.        , ...,  211.88381197,
            0.        ,  132.42738248],
        [   0.        ,   31.62661436,    0.        , ...,  463.49583869,
          132.42738248,    0.        ]])